# DATA CLEANING FINAL PROJECT 
CS513 - Summer 2021 <br>
Anish Saha, Sharanya Balaji, Hung Nguyen

## PART I. Installing Dependencies, Retrieving the Dataset
Here we install the necessary dependencies and retrieve the original dataset **D** as a .csv file from a personal github repository. The NYC dataset was downloaded from: http://insideairbnb.com/get-the-data.html 

In [1]:
import pandas as pd
import numpy as np
import re, io, csv
import requests
import sqlite3

!pip install pyflowchart
from pyflowchart import *

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
CSV_URL = 'https://raw.githubusercontent.com/anish-saha/airbnbNYC_data_pipeline/master/airbnbNYC.csv'
download = requests.get(CSV_URL).content

## PART II. Setting Preliminary Integrity Constraints
Here we run our integrity constraints on the uncleaned data to prove that the conditions for the **U1** use case have not been fulfilled.

In [3]:
conn0 = sqlite3.connect('Test0DB.db')
c0 = conn0.cursor()

In [4]:
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.to_sql('LISTINGSOG', conn0, if_exists='replace', index = False)

In [5]:
c0.execute('''SELECT * FROM Listingsog WHERE minimum_nights < 0 OR minimum_nights > 6 OR maximum_nights < 6''')
len(c0.fetchall())

26312

In [6]:
c0.execute('''SELECT bathrooms FROM Listingsog WHERE bathrooms < 1.5''') # this should show nothing to begin with as well since this is a new column name. previous column returns strings so comparison cannot be made.
len(c0.fetchall())

0

In [7]:
c0.execute(''' SELECT * FROM Listingsog WHERE accommodates < 4 OR beds < 2 OR price > 800.0 ''')
len(c0.fetchall())

28377

In [8]:
c0.execute(''' SELECT * FROM Listingsog WHERE neighbourhood_cleansed != "Financial District" AND neighbourhood_cleansed != "Greenwich Village" AND neighbourhood_cleansed != "Soho" AND neighbourhood_cleansed != "Lower East Side" AND neighbourhood_cleansed != "Chinatown" AND neighbourhood_cleansed != "Hamilton" AND neighbourhood_cleansed = "Clinton" AND neighbourhood_cleansed != "Chelsea" AND neighbourhood_cleansed != "Midtown" AND neighbourhood_cleansed != "Stuyvesant Town" AND neighbourhood_cleansed != "Turtle Bay" AND neighbourhood_cleansed != "Upper West Side" AND neighbourhood_cleansed != "Inwood" AND neighbourhood_cleansed != "Upper East Side" AND neighbourhood_cleansed != "Central Harlem" AND neighbourhood_cleansed != "East Harlem" AND neighbourhood_cleansed != "Morningside Heights" AND neighbourhood_cleansed != "Washington Heights"''') # neighborhoods
len(c0.fetchall())

0

In [9]:
c0.execute('''SELECT * FROM Listingsog WHERE number_of_reviews <=10 OR review_scores_rating <= 80.0''')
len(c0.fetchall())

24488

In [10]:
c0.execute('''SELECT * FROM Listingsog WHERE room_type != "Entire home/apt"''')
len(c0.fetchall())

17662

In [11]:
c0.execute('''SELECT * FROM Listingsog WHERE picture_url = ""''')
len(c0.fetchall())

0

In [12]:
c0.execute('''SELECT * FROM Listingsog WHERE host_identity_verified = "f"''')
len(c0.fetchall())

6918

In [13]:
c0.execute('''SELECT * FROM Listingsog WHERE host_response_time = "within a day"''')
len(c0.fetchall())

2906

## PART III. Data Cleaning Pipeline
In our case, **U1** describes a use case where a user is seeking to take a college trip, so the user is part of a group of 4 friends seeking to stay at a place in New York City for 6 nights within a specific price range (less than $800/night), neighborhood (Manhattan), and is very well-reviewed (at least 10 reviews with an average rating higher than 80/100). In addition, the user would prefer that the place be an entire private setup (without the host), that the host provide at least 1 image (to ensure veracity), and that the place allow smoking or partying. Finally, to ensure that both the host and users involved feel comfortable, users would prefer experienced, verified, hosts with fast response times. A number of extraneous columns are also removed for the sake of convenience; it enhances the customer’s experience as it will be very difficult for a user to comprehend nearly 80 categories when weighing the pros and cons of listings.


In [14]:
st = StartNode('Original Dataset D')
num_rows_original = len(df)
num_cols_original = len(df.columns)

In [15]:
op0 = OperationNode('Filter by number of nights')
result = df.loc[(df['minimum_nights'] > 0) & (df['minimum_nights'] <= 6) & (df['maximum_nights'] >= 6)]
print('Peforming data filtering, as per the user\'s preferences.'); print()
print('Filtering by number of nights: ' + str(num_rows_original-len(result)) + ' rows removed.')
num_rows = len(result)

Peforming data filtering, as per the user's preferences.

Filtering by number of nights: 26312 rows removed.


In [16]:
def convert_bathrooms_text_to_float(x):
  if type(x) == str:
    match = re.findall(r"[-+]?\d*\.\d+|\d+", x)
    if len(match) > 0:
      return float(match[0])
    else:
      return None
  else:
    return None

op1 = OperationNode('Preprocess and clean the bathrooms_text column')
result['bathrooms'] = result['bathrooms_text'].apply(convert_bathrooms_text_to_float)
result = result.loc[result['bathrooms'] >= 1.5]
print('Peforming data filtering, as per the user\'s preferences.'); print()
print('Filtering by number of bathrooms: ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Peforming data filtering, as per the user's preferences.

Filtering by number of bathrooms: 8911 rows removed.


<ipython-input-16-b117a3c47dda>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['bathrooms'] = result['bathrooms_text'].apply(convert_bathrooms_text_to_float)


In [17]:
def convert_price_to_float(x):
  if type(x) == str:
    match = x.replace('$', '')
    match = match.replace(',', '')
    if len(match) > 0:
      return float(match)
    else:
      return -1.0
  else:
    return -1.0

def convert_float_to_price(x):
  return '$' + '{:.2f}'.format(x)

op2 = OperationNode('Filter by price, number of guests, and number of beds')
result['price'] = result['price'].apply(convert_price_to_float)
result = result.loc[(result['accommodates'] >= 4) & (result['beds'] >= 2) & (result['price'] > 0.0) & (result['price'] <= 800.0)]
result['price'] = result['price'].apply(convert_float_to_price) # return the price column to its original formatting

print('Filtering by price, number of guests to accomodate, and number of beds: ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Filtering by price, number of guests to accomodate, and number of beds: 869 rows removed.


In [18]:
op3 = OperationNode('Filter by specified neighborhood (Manhattan)')
neighborhoods = ['Financial District', 'Greenwich Village', 'Soho', 'Lower East Side', 'Chinatown', 'Hamilton',
                 'Clinton', 'Chelsea', 'Midtown', 'Stuyvesant Town', 'Turtle Bay', 'Upper West Side', 'Inwood',
                 'Upper East Side', 'Central Harlem', 'East Harlem', 'Morningside Heights', 'Washington Heights']
result = result[result['neighbourhood_cleansed'].isin(neighborhoods)] # filter by only neighborhoods in Manhattan
print('Filtering by specified neighbourhood: ' + str(num_rows-len(result)) + ' rows removed.')
result['neighborhood'] = result['neighbourhood_cleansed'] # better naming convention, original neighbourhood is uninformative
num_rows = len(result)

Filtering by specified neighbourhood: 671 rows removed.


In [19]:
op4 = OperationNode('Filter by well-reviewed rentals')
result = result.loc[(result['number_of_reviews'] > 10) & (result['review_scores_rating'] > 80.0)]
print('Filtering by well-reviewed rentals: ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Filtering by well-reviewed rentals: 74 rows removed.


In [20]:
op5 = OperationNode('Filter by room type, remove entries for shared rooms, apartments, and homes')
result = result.loc[result['room_type'].str.strip() == 'Entire home/apt'] 
print('Filtering by room type specification (private setups only): ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Filtering by room type specification (private setups only): 6 rows removed.


In [21]:
result = result.loc[result['picture_url'].str.strip() != ''] 
print('Filtering out places with no images: ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Filtering out places with no images: 0 rows removed.


In [22]:
op6 = OperationNode('Add column to flag non-smoking listings based on name/description')
non_smoking_boolean = result.description.str.contains('smok') | result.name.str.contains('smok')
result['no_smoking?'] = np.where(non_smoking_boolean, 'True', 'Not specified')

In [23]:
op7 = OperationNode('Filtering out places with no specification on no smoking')
result = result.loc[result['no_smoking?'].str.strip() != 'True']
print('Filtering out places with no specification on no smoking: ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Filtering out places with no specification on no smoking: 2 rows removed.


In [24]:
op8 = OperationNode('Add column to flag non-party listings based on name/description')
non_party_boolean = result.description.str.contains('no.{0,5}party') | result.name.str.contains('no.{0,5}party')
result['no_party?'] = np.where(non_party_boolean, 'True', 'Not specified')

In [25]:
op9 = OperationNode('Filter out places with no parties allowed')
result = result.loc[result['no_party?'].str.strip() != 'True']
print('Filtering out places with no parties allowed: ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Filtering out places with no parties allowed: 0 rows removed.


In [26]:
op10 = OperationNode('Filtering out hosts that are not verified')
result = result.loc[result['host_identity_verified'].str.strip() != 'f']
print('Filtering out hosts that are not verified: ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Filtering out hosts that are not verified: 9 rows removed.


In [27]:
op11 = OperationNode('Filtering out hosts that don\'t respond within a few hours')
result = result.loc[result['host_response_time'].str.strip() != 'within a day']
print('Filtering out hosts that don\'t respond within a few hours: ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Filtering out hosts that don't respond within a few hours: 7 rows removed.


In [28]:
def is_inexperienced_host(x):
  if type(x) == str:
    matches = re.findall(r"([1-9]|1[0-2])\/([1-9]|1[0-9]|2[0-9]|3[0-1])\/(20|21)", x)
    if len(matches) == 0: 
      return False
    else: 
      return True
  else:
    return True

op11 = OperationNode('Filtering out inexperienced hosts and hosts who typically have slow response times.')
result['inexperienced_host'] = result['host_since'].apply(is_inexperienced_host)
result = result.loc[(result['inexperienced_host'] == False) & ((result['host_response_time'] == 'within an hour') | (result['host_response_time'] == 'within a few hours'))]
print('Filtering out inexperienced hosts who have only been hosts since 2020 or 2021: ' + str(num_rows-len(result)) + ' rows removed.')
num_rows = len(result)

Filtering out inexperienced hosts who have only been hosts since 2020 or 2021: 14 rows removed.


In [29]:
op12 = OperationNode('Remove unnecessary columns')
result = result.drop(columns=['id', 'scrape_id', 'last_scraped', 'host_is_superhost', 'host_url', 'host_about', 'host_listings_count', 'neighbourhood',
                              'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_thumbnail_url', 'host_picture_url', 
                              'host_neighbourhood', 'host_response_rate',	'host_acceptance_rate', 'no_party?', 'no_smoking?', 'inexperienced_host',
                              'neighbourhood_cleansed', 'neighborhood_overview', 'neighbourhood_group_cleansed', 'bathrooms_text', 'listing_url', 
                              'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights','maximum_maximum_nights', 'license',
                              'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'availability_30', 'availability_60', 'property_type', 
                              'availability_90', 'availability_365', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_value',
                              'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'calendar_last_scraped', 
                              'review_scores_communication', 'review_scores_location', 'number_of_reviews_ltm', 'latitude', 'longitude',
                              'instant_bookable',  'reviews_per_month', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
                              'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'has_availability']);
print('Removing unnecessary colums: ' + str(num_cols_original-len(result.columns)) + ' cols removed.')

Removing unnecessary colums: 53 cols removed.


## PART IV. Create an Inner Workflow Model 
Using the ```pyflowchart``` library, we develop an inner workflow model to display the data pipeline that processes the original dataset **D** and generates the final, cleaned dataset **D0**. The output of this code is then copy-pasted into http://flowchart.js.org/ in order to translate the generated textual representation into the SVG flow chart diagram used in the project report. 




In [30]:
# Create Inner Workflow model, use http://flowchart.js.org/ and translate the generated textual representation into SVG flow chart diagrams
e = EndNode('Cleaned Dataset D0')
st.connect(op0)
op0.connect(op1)
op1.connect(op2)
op2.connect(op3)
op3.connect(op4)
op4.connect(op5)
op5.connect(op6)
op6.connect(op7)
op7.connect(op8)
op8.connect(op9)
op9.connect(op10)
op10.connect(op11)
op11.connect(op12)
op12.connect(e)

fc = Flowchart(st)
print(fc.flowchart())

st0=>start: start Original Dataset D
op1=>operation: Filter by number of nights
op2=>operation: Preprocess and clean the bathrooms_text column
op3=>operation: Filter by price, number of guests, and number of beds
op4=>operation: Filter by specified neighborhood (Manhattan)
op5=>operation: Filter by well-reviewed rentals
op6=>operation: Filter by room type, remove entries for shared rooms, apartments, and homes
op7=>operation: Add column to flag non-smoking listings based on name/description
op8=>operation: Filtering out places with no specification on no smoking
op9=>operation: Add column to flag non-party listings based on name/description
op10=>operation: Filter out places with no parties allowed
op11=>operation: Filtering out hosts that are not verified
op13=>operation: Filtering out inexperienced hosts and hosts who typically have slow response times.
op14=>operation: Remove unnecessary columns
e15=>end: end Cleaned Dataset D0

st0->op1
op1->op2
op2->op3
op3->op4
op4->op5
op5->op6


In [31]:
# Create Outer Workflow model, use http://flowchart.js.org/ and translate the generated textual representation into SVG flow chart diagrams
out_st = StartNode('Data Profiling: Manually inspect dataset columns and entries and devise a plan on how to clean dataset for U1')
out_op0 = OperationNode('Data Loading: Load data into Python workspace')
out_io0 =  InputOutputNode(InputOutputNode.INPUT, 'Original dataset: airbnbNYC.csv')
out_op1 = OperationNode('Data Cleaning (Inner Workflow Model)')
out_io1 = InputOutputNode(InputOutputNode.OUTPUT, 'Cleaned dataset: airbnbNYC2021_cleaned.csv')
out_io2 = InputOutputNode(InputOutputNode.INPUT, 'Cleaned dataset: airbnbNYC2021_cleaned.csv')
out_op2 = OperationNode('IC violation checks')
out_e = EndNode('Save cleaned dataset D0')

out_st.connect(out_op0)
out_op0.connect(out_io0)
out_io0.connect(out_op1)
out_op1.connect(out_io1)
out_io1.connect(out_io2)
out_io2.connect(out_op2)
out_op2.connect(out_e)

out_fc = Flowchart(out_st)
print(out_fc.flowchart())


st17=>start: start Data Profiling: Manually inspect dataset columns and entries and devise a plan on how to clean dataset for U1
op18=>operation: Data Loading: Load data into Python workspace
io19=>inputoutput: input: Original dataset: airbnbNYC.csv
op20=>operation: Data Cleaning (Inner Workflow Model)
io21=>inputoutput: output: Cleaned dataset: airbnbNYC2021_cleaned.csv
io22=>inputoutput: input: Cleaned dataset: airbnbNYC2021_cleaned.csv
op23=>operation: IC violation checks
e24=>end: end Save cleaned dataset D0

st17->op18
op18->io19
io19->op20
op20->io21
io21->io22
io22->op23
op23->e24



## Part V. Generate CSV Output
In this step we generate an output .csv file for **D0**, our cleaned dataset.

In [32]:
result.to_csv("./airbnbNYC2021_cleaned.csv")

## Part VI. Checking Integrity Constraints
The final step is to run our integrity constraints on the cleaned data to prove that the conditions for the **U1** use case have been successfully fulfilled. As can be seen from the fact that all of the SQL queries in our integrity constraints now return 0 rows, our data cleaning pipeline has functioned as originally intended.


In [33]:
conn = sqlite3.connect('TestDB.db')
c = conn.cursor()
read_listings = pd.read_csv("./airbnbNYC2021_cleaned.csv")
read_listings.to_sql('LISTINGS', conn, if_exists='replace', index = False)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [34]:
c.execute('''SELECT * FROM Listings WHERE name in (SELECT l1.name FROM Listings l1 JOIN Listings l2 on l1.name = l2.name AND l1.description != l2.description)''')
len(c.fetchall())

0

In [35]:
c.execute('''SELECT * FROM Listings WHERE host_id IN (SELECT l1.host_id FROM Listings l1 JOIN Listings l2 on l1.host_id = l2.host_id AND l1.host_name != l2.host_name)''')
len(c.fetchall())

0

In [36]:
c.execute('''SELECT * FROM Listings WHERE minimum_nights < 0 OR minimum_nights > 6 OR maximum_nights < 6''')
len(c.fetchall())

0

In [37]:
c.execute('''SELECT bathrooms FROM Listings WHERE bathrooms < 1.5''')
len(c.fetchall())

0

In [38]:
c.execute(''' SELECT * FROM Listings WHERE accommodates < 4 OR beds < 2 OR price > 800.0 ''')
len(c.fetchall())

0

In [39]:
c.execute(''' SELECT * FROM Listings WHERE neighborhood != "Financial District" AND neighborhood != "Greenwich Village" AND neighborhood != "Soho" AND neighborhood != "Lower East Side" AND neighborhood != "Chinatown" AND neighborhood != "Hamilton" AND neighborhood = "Clinton" AND neighborhood != "Chelsea" AND neighborhood != "Midtown" AND neighborhood != "Stuyvesant Town" AND neighborhood != "Turtle Bay" AND neighborhood != "Upper West Side" AND neighborhood != "Inwood" AND neighborhood != "Upper East Side" AND neighborhood != "Central Harlem" AND neighborhood != "East Harlem" AND neighborhood != "Morningside Heights" AND neighborhood != "Washington Heights"''') # neighborhoods
len(c.fetchall())

0

In [40]:
c.execute('''SELECT * FROM Listings WHERE number_of_reviews <=10 OR review_scores_rating <= 80.0''')
len(c.fetchall())

0

In [41]:
c.execute('''SELECT * FROM Listings WHERE room_type != "Entire home/apt"''')
len(c.fetchall())

0

In [42]:
c.execute('''SELECT * FROM Listings WHERE picture_url = ""''')
len(c.fetchall())

0

In [43]:
c.execute('''SELECT * FROM Listings WHERE host_identity_verified = "f"''')
len(c.fetchall())

0

In [44]:
c.execute('''SELECT * FROM Listings WHERE host_response_time = "within a day"''')
len(c.fetchall())

0